<a href="https://colab.research.google.com/github/DesmondWasHere/colab-scraping/blob/main/TOI_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Installing Selnium and Chromium**

In [ ]:
## Downloading Selenium and chrome webdriver.
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# **Step 2: Importing the header file and setting system path for chromedriver**

In [2]:
## Header Files
import sys
import csv
import time
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup 
from selenium import webdriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# **Step 3: Selnium Settings for colab**

In [3]:
## Selnium Custom Settings for colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


# **Step 4: Run it for json format.**

In [ ]:
## Year and month to scrape
year = input("Input the year you want to scrape from, eg. 2015: ")
month =input("Input the month you want to scrape from, eg. 2: ")
particularDay = int(input("Input the day you want to scrape from, eg. 1: "))
particularEnd = int(input("Input the day you want to scrape till, eg. 10: "))
articlePerDay = int(input("Enter the number of articles per day (Optimal 10):"))

## Opening the link
url = "https://timesofindia.indiatimes.com/archive/year-{},month-{}.cms".format(year,month)
driver.get(url)

## Generating the list of links of day
dayList = []
for x in range(2,7):
    for y in range(1,8):
        try:
            day = driver.find_element_by_xpath("/html/body/center/div[1]/div[3]/table[2]/tbody/tr[2]/td[1]/div[3]/table/tbody/tr[1]/td/span/div/table/tbody/tr[{}]/td[{}]/a".format(x,y))
            dayList.append(day.get_attribute('href'))
        except:
            continue
## Opening a csv file, scraping and storing the values.
fileName = 'TOIArticleArchive.csv'
with open(fileName, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Date","Author","Vertical","Headline","Description"])
    for d in range(particularDay,particularEnd+1):
      for day in dayList:
          if (dayList.index(day) != d-1):
            print("Skipping day", dayList.index(day)+1)
            continue
          ## iterate in dayList
          print("\n Running on day",day)
          articleLinks = []
          count = 1

          r = requests.get(day)

          soup = BeautifulSoup(r.content, 'html5lib')
          for a in soup.find_all('tbody')[2].find_all('a',href=True):
              if a.text:
                  articleLinks.append(a['href'])
          print("Articles Found",len(articleLinks))
          if(len(articleLinks)== 0):
              print("No Articles Here")
              continue
          ## iterate in article Links
          count = 1
          for link in range(0,len(articleLinks)):
            if count>articlePerDay:
                break
            try:
              print("Scraping article {} of {}".format(count, articlePerDay))
              # Opening an article
              try:
                r = requests.get(articleLinks[link])
              except:
                print("Website error, ",articleLinks[link])
                continue
              soup = BeautifulSoup(r.content, 'html5lib')
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                day = soup.find('div',attrs = {'class':'as_byline'}).find('div',attrs = {'data-plugin':"dateformat"}).text[9:]
              except:
                try:
                  dayStore = soup.find('div',attrs = {'class':'_3Mkg- byline'}).text
                  day = dayStore[dayStore.rfind("|")+1:]
                except:
                  dayStore = soup.find('div',attrs = {'class':'yYIu- byline'}).text
                  day = dayStore[dayStore.rfind("/")+1:]

              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                author = soup.find('a',attrs = {'class':'auth_detail'}).text ## code for original
              except:
                try:
                  author = soup.find('div',attrs = {'class':'as_byline'}).find('div').text ## code for original
                except:
                  try:
                    author = soup.find('div',attrs = {'class':'_3Mkg- byline'}).find('span').text
                  except:
                    try:
                      dayStore = soup.find('div',attrs = {'class':'_3Mkg- byline'}).text
                      author = dayStore[:dayStore.find("|")]
                    except:
                      dayStore = soup.find('div',attrs = {'class':'yYIu- byline'}).text
                      author = dayStore[:dayStore.find("/")]
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              vertical = (" ".join(articleLinks[link].split("/")[5:len(articleLinks[0].split("/"))-3]))
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              headline = soup.find('title').text
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              try:
                article = soup.find("arttextxml").text
              except:
                try:
                  article = soup.find("Normal").text
                except:
                  try:
                    article = soup.find('div',attrs = {'class':'ga-headlines'}).text
                  except:
                    try:
                      article = soup.find('div',attrs = {'class':'Normal'}).text
                    except:
                      article = soup.find('div',attrs = {'class':'_3YYSt clearfix '}).text
              #~~~~~~~~~~~~~~~~~~~~~~~~~~~
              writer.writerow([day.encode(encoding='UTF-8',errors='ignore'),author.encode(encoding='UTF-8',errors='ignore'),vertical.encode(encoding='UTF-8',errors='ignore'),headline.encode(encoding='UTF-8',errors='ignore'),article.encode(encoding='UTF-8',errors='ignore')])
              count+=1
            except:
              print("Skipping ",articleLinks[link],"Unknown Format")
              # pass
          break


# **Step 5: Run it for xml and json format. (Run Step 4 first.)**

In [28]:
csvFile = fileName
xmlFile = fileName.replace(".csv",".xml")
jsonFile = fileName.replace(".csv",".json")

df = pd.read_csv (fileName)
df.to_json (jsonFile)

csvData = csv.reader(open(csvFile,encoding='UTF-8',errors='ignore'))
xmlData = open(xmlFile, 'w')
xmlData.write('<?xml version="1.0"?>' + "\n")
# there must be only one top-level tag
xmlData.write('<csv_data>' + "\n")

rowNum = 0
for row in csvData:
    if rowNum == 0:
        tags = row
        # replace spaces w/ underscores in tag names
        for i in range(len(tags)):
            tags[i] = tags[i].replace(' ', '_')
    else:
        xmlData.write('<row>' + "\n")
        for i in range(len(tags)):
            xmlData.write('    ' + '<' + tags[i] + '>' \
                          + row[i] + '</' + tags[i] + '>' + "\n")
        xmlData.write('</row>' + "\n")

    rowNum +=1

xmlData.write('</csv_data>' + "\n")
xmlData.close()
